# NMF - Negative Matrix Factorization

by Sergiu Netotea, NBIS

### Recommender systems 1.
**How can I explain multi-omics integration to a rich guy?**

> "Research is a derivative of a society's general drift, rarely the opposite." :) personal view, reader discretion is advised!

- Simple definition: Reccomend an item to a user. Existentialists unite! Are we a user or an item? Is a complex disease best described by phenotipic effects or multi-omics causes?
- Rich guys have THE BEST data. Here are some example data tables:
    - user/likes section: users/films, users/pictures, users/groceries, users/houses, users/profiles
    - item/usage section: item/construction costs, item/delivery options, item/user rankings
    - tax avoidance/country section: ... etc
- If reccomender systems research had the same budget as multi-omics research:


<img src="img/budget.jpg" width="200"/>


### Recommender systems 2. Collaborative filtering


| User CF             |  Item CF |
:--------------------:|:-------------------------:
<img src="img/user.png" width="200"/>  | <img src="img/item.png" width="200"/> 


- Users who share the same interest in certain kind of items will also share the same interest in some other kind of items.
- Each user (sample) can be described by k attributes or features. Example: How likely is it that a person suffers from a certain type of cancer?
- Each item (gene) can be described by an analagous set of k attributes or features. Example: How likely is it for the gene to be involved/co-regulated/induced etc in a certain type of cancer.
- We don't know what these features are, how many are relevant. We learn them, or rather let the machines pick.


### Reccomender systems 3. - Matrix factorization

<img src="img/NMF.png" width="300"/>

Hidden representation of a user u predicted rating for item i based on k latent factors (hidden features):
- $r_{ui} = x_u y_i^T = \sum_k{x_{uk} y_{ki}}$
- Fit function: $ L = \sum_{u,i \in S} (r_{ui} - x_u y_i^T)^2$
- Many methods exist:
    - alternating least squares (ALS)
    - stochastic gradient descent (SGD)
- For an *ab-initio* example on how to perform general matrix factorization using 2 methods:
    - https://blog.insightdatascience.com/explicit-matrix-factorization-als-sgd-and-all-that-jazz-b00e4d9b21ea

### Reccomender systems 4. - NMF

<img src="img/segmentation.png" width="200"/>

General model assumption: The higher the factor weight, the more “determined” the column (segment) is by the variable in the row. Ex: Intensity based omics measurements.

Exercise: Perform the collaborative filtering in the following article
- https://medium.com/logicai/non-negative-matrix-factorization-for-recommendation-systems-985ca8d5c16c

Further read:
- NMF outperforms similar techniques at learning parts representation:
    - Lee, D., Seung, H. Learning the parts of objects by non-negative matrix factorization. Nature 401, 788–791 (1999). https://doi.org/10.1038/44565

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

V = np.array([[0,1,0,1,2,2],
              [2,3,1,1,2,2],
              [1,1,1,0,1,1],
              [0,2,3,4,1,1],
              [0,0,0,0,1,0]])

V = pd.DataFrame(V, columns=['John', 'Alice', 'Mary', 'Greg', 'Peter', 'Jennifer'])
V.index = ['Vegetables', 'Fruits', 'Sweets', 'Bread', 'Coffee']

nmf = NMF(3)
nmf.fit(V)

H = pd.DataFrame(np.round(nmf.components_,2), columns=V.columns)
H.index = ['Fruits pickers', 'Bread eaters',  'Veggies']

W = pd.DataFrame(np.round(nmf.transform(V),2), columns=H.index)
W.index = V.index

reconstructed = pd.DataFrame(np.round(np.dot(W,H),2), columns=V.columns)
reconstructed.index = V.index

## NMF - algorithms

- Chen et al, Attention-Based Multi-NMF Deep Neural Network with Multimodality Data for Breast Cancer Prognosis Model, Volume 2019 |Article ID 9523719 | https://doi.org/10.1155/2019/9523719

- multiplicative update algorithms
- Alternating Least Square
- NMF based on Optimal Brain Surgery and Alternate Least Square algorithms
- NMF based on projection gradient algorithms
- PNMF probabilistic nonnegative matrix factorization



# jNMF

- Fujita, N., Mizuarai, S., Murakami, K. et al. Biomarker discovery by integrated joint non-negative matrix factorization and pathway signature analyses. Sci Rep 8, 9743 (2018). https://doi.org/10.1038/s41598-018-28066-w

Main goals:
- biomarker discovery
- is theoretically and practically equivalent to a standard NMF method with concatenated inputs
- common clusters (co-modules) from mRNA expression, microRNA expression, and DNA methylation data of cancer patients

<img src="img/jnmf_pharma.png" width="500"/>


### JNMF - method details

- objective function: $\sum_{i=1}^N \|X_i - WH_i\|$
- NMF method was modified to deal with missingness, by using a mask

# SNMF - Sparse NMF

- Yuan Gao, George Church, Improving molecular cancer class discovery through sparse non-negative matrix factorization, Bioinformatics, Volume 21, Issue 21, , Pages 3970–3975, https://doi.org/10.1093/bioinformatics/bti653
- Brunet et al., 2004, Metagenes and molecular pattern discovery using matrix factorization
PNAS, Mar 2004, 101 (12) 4164-4169; https://doi.org/10.1073/pnas.0308531101

- cancer subtyping in microarrays, dealing with sparse data
- not integrative (but does that *really* matter?)
- rank estimation of NMF models
- the representation learned is sparse and hierarchical
- $A \approx WH$ The more sparse the matrix of H, the more sparse is the feature matrix W. Therefore, enforcing the sparseness of H will give rise to metagenes that comprised few dominantly deterministic genes.

## Deep MF

- Different NN setups exist for almost every type of MF!
- Example: Supervised deep matrix factorization on the MovieLens dataset, using mxnet:
    - https://www.oreilly.com/content/deep-matrix-factorization-using-apache-mxnet/

Advantages:
- Take advantage of NN advancements such as mixing batch normalization layers to speed up training time.
- Restrict the size of user/item factors independently. Useful in the case where one dimension may be significantly larger than the other and, thus, requires training a massive number of factors

<img src="img/deep_mf.png" width="200"/>


In [ ]:
X_train = mx.io.NDArrayIter({'user': train_users, 'movie': train_movies, 'movie_genre': train_genres}, 
                            label=train_ratings, batch_size=batch_size)
X_eval = mx.io.NDArrayIter({'user': valid_users, 'movie': valid_movies, 'movie_genre': valid_genres}, 
                           label=valid_ratings, batch_size=batch_size)

user = mx.symbol.Variable("user")
user = mx.symbol.Embedding(data=user, input_dim=n_users, output_dim=15)

movie = mx.symbol.Variable("movie")
movie = mx.symbol.Embedding(data=movie, input_dim=n_movies, output_dim=20) # Reduce from 25 to 20

# We need to add in a third embedding layer for genre
movie_genre = mx.symbol.Variable("movie_genre")
movie_genre = mx.symbol.Embedding(data=movie_genre, input_dim=20, output_dim=5) # Set to 5

y_true = mx.symbol.Variable("softmax_label")

nn = mx.symbol.concat(user, movie, movie_genre)
nn = mx.symbol.flatten(nn)
nn = mx.symbol.FullyConnected(data=nn, num_hidden=64)
nn = mx.symbol.Activation(data=nn, act_type='relu')
nn = mx.symbol.FullyConnected(data=nn, num_hidden=64)
nn = mx.symbol.Activation(data=nn, act_type='relu')
nn = mx.symbol.FullyConnected(data=nn, num_hidden=1)

y_pred = mx.symbol.LinearRegressionOutput(data=nn, label=y_true)

model = mx.module.Module(context=mx.gpu(0), data_names=('user', 'movie', 'movie_genre'), symbol=y_pred)
model.fit(X_train, num_epoch=5, optimizer='adam', optimizer_params=(('learning_rate', 0.001),),
          eval_metric='rmse', eval_data=X_eval, batch_end_callback=mx.callback.Speedometer(batch_size, 250))

## Deep NMF

- Flenner, J., & Hunter, B. (2017). A Deep Non-Negative Matrix Factorization Neural Network.
    - https://www1.cmc.edu/pages/faculty/BHunter/papers/deep-negative-matrix.pdf


- Normal NMF is performed by a single encoding layer 

<img src="img/nmf_onelayer.png" width="200"/>


- Deep architecture: CNN, with backpropagation, each NMF layer performs a hierarchical decomposition

<img src="img/nmf_multilayered.png" width="200"/>


## Matrix Factorization - beyond NMF

- Scluster method (a mix of MF and SNF): integrates different types of data and maps them into an effective low-dimensional subspace. First, Scluster uses adaptive sparse reduced-rank regression (S-rrr) to map the original data into the principal subspaces. Next, a fused patient-by-patient network is abstracted for these subgroups by a scaled exponential similarity kernel method. It can then obtain the cancer subtypes by spectral clustering.
    - https://pubmed.ncbi.nlm.nih.gov/28113782/
- SRF : rank based multi-view bi-clustering, very related to NMF, possibly reducible to it, it does subtyping and identification of subtype-specific features simultaneously.
    - https://pubmed.ncbi.nlm.nih.gov/27587661/

Software:
- Python reccomender systems library: http://surpriselib.com/
- R joint NNF library: https://github.com/rikenbit/nnTensor/
- Deep Semi NMF: https://github.com/trigeorgis/Deep-Semi-NMF


Rank estimation of NMF:
- Jean-Philippe Brunet. et. al., (2004). Metagenes and molecular pattern discovery using matrix factorization. PNAS
- Xiaoxu Han. (2007). CANCER MOLECULAR PATTERN DISCOVERY BY SUBSPACE CONSENSUS KERNEL CLASSIFICATION
- Attila Frigyesi. et. al., (2008). Non-Negative Matrix Factorization for the Analysis of Complex Gene Expression Data: Identification of Clinically Relevant Tumor Subtypes. Cancer Informatics
- Haesun Park. et. al., (2019). Lecture 3: Nonnegative Matrix Factorization: Algorithms and Applications. SIAM Gene Golub Summer School, Aussois France, June 18, 2019
- Chunxuan Shao. et. al., (2017). Robust classification of single-cell transcriptome data by nonnegative matrix factorization. Bioinformatics


Other integrative NMF:


- intNMF: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0176278
- Simultaneous Non-negative Matrix Factorization (siNMF) : Extracting Gene Expression Profiles Common to Colon and Pancreatic Adenocarcinoma using Simultaneous nonnegative matrix factorization, Liviu Badea, Pacific Symposium on Biocomputing, 13:279-290, 2009, 
- Discovery of multi-dimensional modules by integrative analysis of cancer genomic data. Shihua Zhang, et al., Nucleic Acids Research, 40(19), 9379-9391, 2012
- Probabilistic Latent Tensor Factorization, International Conference on Latent Variable Analysis and Signal Separation, Y. Kenan Yilmaz et al., 346-353, 2010


Fast Tensorial Calculus:


- Non-negative Matrix Factorization (NMF) : [Nonnegative Matrix and Tensor Factorizations, Andrzej CICHOCK, et. al., 2009](https://pdfs.semanticscholar.org/94cc/6daad548a03c6edb0351d686c2d4aa364634.pdf), 
- [A Study on Efficient Algorithms for Nonnegative Matrix/Tensor Factorization, Keigo Kimura, 2017](https://eprints.lib.hokudai.ac.jp/dspace/bitstream/2115/65379/1/Keigo_Kimura.pdf)

Non-negative CP Decomposition (NTF)
   - α-Divergence (KL, Pearson, Hellinger, Neyman) / β-Divergence (KL, Frobenius, IS) : [Non-negative Tensor Factorization using Alpha and Beta Divergence, Andrzej CICHOCKI et. al., 2007](http://mlg.postech.ac.kr/static/publications/inter_conf/2007/icassp07_cichocki.pdf), [TensorKPD.R (gist of mathieubray)](https://gist.github.com/mathieubray/d83ce9c13fcb60f723f957c13ad85ac5)
   - Fast HALS : [Multi-way Nonnegative Tensor Factorization Using Fast Hierarchical Alternating Least Squares Algorithm (HALS), Anh Huy PHAN et. al., 2008](http://www.ieice.org/proceedings/NOLTA2008/articles/A1L-D3-Phan-2045.pdf)
   - α-HALS/β-HALS : [Fast Local Algorithms for Large Scale Nonnegative Matrix and Tensor Factorizations, Andrzej CICHOCKI et. al., 2008](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.214.6398&rep=rep1&type=pdf)
- Non-negative Tucker Decomposition (NTD)
   - KL, Frobenius : [Nonnegative Tucker Decomposition, Yong-Deok Kim et. al., 2007](https://pdfs.semanticscholar.org/f388/99be8ebd8b9aa7029b2b4f187dac4b04d816.pdf)
   - α-Divergence (KL, Pearson, Hellinger, Neyman) / β-Divergence (KL, Frobenius, IS) : [Nonneegative Tucker Decomposition With Alpha-Divergence, Yong-Deok Kim et. al., 2008](https://pdfs.semanticscholar.org/f01b/7354619f053863048217c58cc517def86aeb.pdf), [Fast and efficient algorithms for nonnegative Tucker decomposition, Anh Huy Phan, 2008](https://link.springer.com/chapter/10.1007/978-3-540-87734-9_88)
   - Fast HALS : [Extended HALS algorithm for nonnegative Tucker decomposition and its applications for multiway analysis and classification, Anh Hyu Phan et. al., 2011](https://www.sciencedirect.com/science/article/pii/S0925231211000427)